### IEX Code -------------------------------------------------------
1. Define the stock attributes
2. Define functions: filter attributes, fetch stock data, log events
3. Read company list from the redis datastore
4. Setup schedule
5. Fetch data at scheduled event.

In [6]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime
import traceback
import os

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [ ]:
########### Set up Feature Flag #####################

   
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [14]:
############ Define Functions #################################
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]

#Select interesting stock attributes.
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output


#Create function to fetch the stock data. This is to prepare for the schedule.

def fetch_stock_data(stocks=[]):
    try:
        #Only pull stock info if the DataFlag is set.
        if int(REDIS.get('Data_On')) == 1:

            for ticker in stocks:
                url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
                response = urllib.request.urlopen(url)
                str_response = response.read().decode('utf-8')
                obj = json.loads(str_response)
                filtered = filter_stock_attributes(obj)
                #Add counter to count stocks. 
                REDIS.incr('IEX_Stock_Count')
                #<----- Insert to Kinesis Stream ------->
                if int(get_feature_flag('kinesis_stream_write'))==1:
                    response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                print('---------------------------------')
                #print(response)
                print(filtered)
    except:
        print(traceback.format_exc())
        #Send Error event
        send_event('IEX', 'Error', 'Error occured, check stdout')

#Setup Log

global past_stock_count
past_stock_count = 0




########### Redis Subscription setup ########################
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')


#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
    


#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Send the log event
    send_log(source='IEX',current_count = current_stock_count, count_diff=stock_count_diff)
    
    print('Logged Data')

In [15]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(35).seconds.do(log)

#Send Start event
send_event('IEX', 'Activity', 'Data Source Started')

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'Apple Inc.', 'latestPrice': 172.43, 'latestVolume': 40047623, 'marketCap': 874912061590, 'open': 172.4, 'previousClose': 172.43, 'sector': 'Technology', 'high': 174.82, 'low': 171.77, 'ytdChange': 0.0009868802972252172, 'peRatio': 18.74, 'week52High': 180.1, 'week52Low': 135.28}
---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'Facebook Inc.', 'latestPrice': 177.36, 'latestVolume': 20657593, 'marketCap': 515230934439, 'open': 178.99, 'previousClose': 177.36, 'sector': 'Technology', 'high': 179.88, 'low': 176.3, 'ytdChange': -0.022379010031969872, 'peRatio': 28.84, 'week52High': 195.32, 'week52Low': 132.9}
---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1094.8, 'latestVolume': 1680258, 'marketCap': 761211735844, 'open': 1088.41, 'previousClose': 1094.8, 'sector': 'Technology', 'high': 11

---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'General Electric Company', 'latestPrice': 15.05, 'latestVolume': 64629453, 'marketCap': 130514879250, 'open': 14.85, 'previousClose': 15.05, 'sector': 'Industrials', 'high': 15.23, 'low': 14.81, 'ytdChange': -0.16295884315906561, 'peRatio': 14.33, 'week52High': 30.59, 'week52Low': 14.23}
---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'UnitedHealth Group Incorporated (DE)', 'latestPrice': 229.37, 'latestVolume': 3617618, 'marketCap': 221952843731, 'open': 225.99, 'previousClose': 229.37, 'sector': 'Healthcare', 'high': 230.55, 'low': 225.7, 'ytdChange': 0.03712244528847897, 'peRatio': 22.75, 'week52High': 250.79, 'week52Low': 157.52}
---------------------------------
{'latestUpdate': '2018-02-16 00:00:00', 'companyName': 'The Priceline Group Inc.', 'latestPrice': 1885, 'latestVolume': 559157, 'marketCap': 91930594210, 'open': 1874.62, 'previousClose': 1885,

KeyboardInterrupt: 